# Generate Data for Chartbook

Brian Dew

@bd_econ

In [1]:
import sys
sys.path.append('../src')

import time

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### S&P 500 

In [2]:
ltdate = int(time.time())

url = ('https://query1.finance.yahoo.com/v7/finance/download/%5EGSPC?'+
       f'period1=599616000&period2={ltdate}&interval=1d&events=history')

df = pd.read_csv(url).set_index('Date')

close = df['Adj Close']

close.index = pd.to_datetime(close.index)

final = close.resample('MS').mean().append(pd.Series({close.index[-1]: close[-1]}))
final.name = 'GSPC'
final.to_csv(data_dir / 'sp500.csv', index_label='date', float_format='%g') 

data = pd.DataFrame()
data['GSPC'] = close.loc['2017':]
data['MA'] = close.rolling(253).mean().loc['2017':]
data.to_csv(data_dir / 'sp500_recent.csv', index_label='date', float_format='%g')

In [3]:
ltdate = dtxt(close.index[-1])['day1']
ltval = close.iloc[-1]
hival = df.High.iloc[-253:].max()
chhi = (1 - ltval/hival) * 100
hidt = dtxt(df.High.iloc[-253:].idxmax())['day1']
loval = df.Low.iloc[-253:].min()
chlo = abs((1 - ltval/loval) * 100)
lodt = dtxt(df.Low.iloc[-253:].idxmin())['day1']
avval = df['Adj Close'].iloc[-253:].mean()
chav = (1 - ltval/avval) * 100
chavabs = abs(chav)
chtype = 'below' if chav > 0 else 'above'

text = (f'The S\&P 500 closed at {ltval:.0f} on {ltdate}. The index is currently '+
        f'{chhi:.1f} percent below its one-year high of {hival:.0f} on {hidt}, and '+
        f'{chlo:.1f} percent above its one-year low of {loval:.0f} on {lodt}. The '+
        f'average over the past year is {avval:.0f}; the index is {chavabs:.1f} percent '+
        f'{chtype} its one-year moving average '+
         '(see {\color{black!18!white}\\textbf{---}}).')

write_txt(text_dir / 'sp500.txt', text)

text

'The S\\&P 500 closed at 2762 on April 13, 2020. The index is currently 18.6 percent below its one-year high of 3394 on February 19, 2020, and 26.0 percent above its one-year low of 2192 on March 23, 2020. The average over the past year is 2987; the index is 7.5 percent below its one-year moving average (see {\\color{black!18!white}\\textbf{---}}).'

### Interest rates

In [9]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H15&series=398c5ba6279c57e57055c6a65c95e9d3&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d = {
    'RIFLGFCY10_N.B': 'Ten-year',
    'RIFSPFF_N.B': 'Fed Funds',
    'RIFLGFCM03_N.B': 'Three-month',
    'RIFLGFCY02_N.B': 'Two-year'
}

df = pd.read_csv(url, skiprows=5, index_col=0)[d.keys()].rename(d, axis=1)

df.index = pd.to_datetime(df.index)

df = df[df['Ten-year'] != 'ND']

data = (df[df['Ten-year'] != 'ND'].astype('float')
        .resample('M').mean().iloc[:-1]
        .append(df.iloc[-1]))

data.to_csv(data_dir / 'rates.csv', index_label='date', float_format='%g')

ldate = dtxt(data.index[-1])['day1']
tenlt = data['Ten-year'].iloc[-1]
tenpr = data['Ten-year'].iloc[-13]
twolt = data['Two-year'].iloc[-1]
twopr = data['Two-year'].iloc[-13]

text = (f'As of {ldate}, the constant maturity yield for a ten-year '+
        'Treasury bond (see {\color{red!70!purple!90!black}\\textbf{---}}) '+
        f'is {tenlt} percent, compared to {tenpr:.2f} percent one year prior. '+
        'The yield for a two-year Treasury (see {\color{orange!80!yellow}\\textbf{---}}) '
        +f'is {twolt} percent, compared to {twopr:.2f} '+
        f'percent a year prior.')

write_txt(text_dir / 'rates_basic.txt', text)

print(text)

text = ('The effective fed funds rate (see {\color{blue!60!black}\\textbf{---}}) '+
        f'is {data["Fed Funds"].iloc[-1]} percent, as of {ldate}.')

write_txt(text_dir / 'rates_ff.txt', text)

text

As of April 9, 2020, the constant maturity yield for a ten-year Treasury bond (see {\color{red!70!purple!90!black}\textbf{---}}) is 0.73 percent, compared to 2.53 percent one year prior. The yield for a two-year Treasury (see {\color{orange!80!yellow}\textbf{---}}) is 0.23 percent, compared to 2.34 percent a year prior.


'The effective fed funds rate (see {\\color{blue!60!black}\\textbf{---}}) is 0.05 percent, as of April 9, 2020.'

### Spread

In [10]:
data = df[df['Ten-year'] != 'ND'].astype('float')
spread = pd.DataFrame()
spread['Ten-3M'] = data['Ten-year'] - data['Three-month']
spread['Ten-2Y'] = data['Ten-year'] - data['Two-year']
spread.loc['2012':].to_csv(data_dir / 'spread.csv', index_label='date', float_format='%g', header=True)

node = end_node(spread['Ten-3M'], 'cyan!60!blue').replace('\\%', '')
write_txt(text_dir / 'spread_node.txt', node)

node = end_node(spread['Ten-2Y'], 'orange').replace('\\%', '')
write_txt(text_dir / 'spread_node2.txt', node)

lt3 = spread['Ten-3M'].iloc[-1]
pr3 = spread['Ten-3M'].iloc[-252]
lt2 = spread['Ten-2Y'].iloc[-1]
pr2 = spread['Ten-2Y'].iloc[-252]

text = (f'As of {ldate}, the spread between a 10-year treasury bond and '+
        f'a three-month treasury bill is {lt3:.2f} percentage points '+
        '(see {\color{cyan!60!blue}\\textbf{---}}), compared to '+
        f'{pr3:.2f} percentage points one year prior. The spread between 10-year '+
        'and 2-year treasuries (see {\color{orange}\\textbf{---}}) '+
        f'was {lt2:.2f} percentage points on {ldate}, and '+
        f'{pr2:.2f} percentage points one year prior.')

write_txt(text_dir / 'spread_basic.txt', text)

text

'As of April 9, 2020, the spread between a 10-year treasury bond and a three-month treasury bill is 0.48 percentage points (see {\\color{cyan!60!blue}\\textbf{---}}), compared to 0.09 percentage points one year prior. The spread between 10-year and 2-year treasuries (see {\\color{orange}\\textbf{---}}) was 0.50 percentage points on April 9, 2020, and 0.16 percentage points one year prior.'

### Fed Balance Sheet

In [11]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H41&series=38d757c01cb3b550176f371352643679&lastobs=&'
dt = 'from=01/01/2002&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

raw_data = pd.read_csv(url)

d = {v: re.sub("\s+[\(\[].*?[\)\]]", "", i.split(';')[0]) 
     for i, v in raw_data.iloc[4, 1:].iteritems()}

date_column = raw_data.loc[5:, 'Series Description']
date_index = pd.to_datetime(date_column).rename('Date')
columns = raw_data.iloc[4, 1:].values
    
clean_data = raw_data.iloc[5:, 1:].replace('ND', np.nan).astype('float')
clean_data.index = date_index
clean_data.columns = columns

clean_data = clean_data / 1_000_000

clean_data.to_csv(data_dir / 'fed_assets.csv', index_label='date', float_format='%g')

In [12]:
ltval = clean_data['RESPPMA_N.WW'].iloc[-1].round(1)
ltdate = dtxt(clean_data.index[-1])['day1']
n = (clean_data.iloc[-1] - clean_data.iloc[-2])['RESPPMA_N.WW'] * 1000
chval = (f'increased by \${abs(n):.0f} billion' if n >= 10 
         else f'decreased by \${abs(n):.0f} billion' 
         if n <= -10 else 'was largely unchanged')

text = ('In response to the collapse of the housing bubble, the Fed purchased '+
        'U.S. Treasury bonds and mortgage-backed securities. Total assets held by '+
        'the Federal Reserve (see {\color{blue!80!black}\\textbf{---}}) '+
        'increased from \$0.9 trillion in August 2008 to '+
        '\$2.2 trillion in November 2008. Additional rounds of asset purchases, '+
        'referred to as quantitative easing, increased the balance sheet to '+
        '\$4.5 trillion by January 2014. As bonds mature they were replaced '+
        'until October 2017, when the Fed allowed the size of its balance '+
        'sheet to normalize. Total assets fell below \$3.8 trillion in August 2019.')

write_txt(text_dir / 'fed_assets1.txt', text)
        
        
txt2 = ('Balance sheet normalization ended in September 2019 when the Fed increased '+
        'operations in overnight and term repurchase agreement (repo) markets, following '+
        'a sharp increase in rates in these markets. The Fed balance sheet '+
        'increased to \$4.1 trillion by December 2019. More recently, in response to '+
        'worsening financial conditions, the Fed began to purchase commercial bonds '+
        'and to offer currency swaps with major US trading partners. '+
        'The Fed balance sheet increased from \$4.2 trillion in February 2020 to '+
        f'\${ltval} trillion, as of the latest data, covering {ltdate}. Fed total assets '+
        f'{chval} over the previous week value.')

write_txt(text_dir / 'fed_assets2.txt', txt2)

print(txt2)

Balance sheet normalization ended in September 2019 when the Fed increased operations in overnight and term repurchase agreement (repo) markets, following a sharp increase in rates in these markets. The Fed balance sheet increased to \$4.1 trillion by December 2019. More recently, in response to worsening financial conditions, the Fed began to purchase commercial bonds and to offer currency swaps with major US trading partners. The Fed balance sheet increased from \$4.2 trillion in February 2020 to \$6.1 trillion, as of the latest data, covering April 8, 2020. Fed total assets increased by \$272 billion over the previous week value.


### Exchange rates

In [13]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H10&series=ad1712193ad5bad7b424e3ae5eb101a5&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

raw_data = pd.read_csv(url)

d = {v: re.sub("\s+[\(\[].*?[\)\]]", "", i.split(';')[0]) 
     for i, v in raw_data.iloc[4, 1:].iteritems()}

date_column = raw_data.loc[5:, 'Series Description']
date_index = pd.to_datetime(date_column).rename('Date')
columns = raw_data.iloc[4, 1:].values
    
clean_data = raw_data.iloc[5:, 1:].replace('ND', np.nan).astype('float')
clean_data.index = date_index
clean_data.columns = columns

clean_data['RXI_N.B.EU'] = 1 / clean_data['RXI$US_N.B.EU'] 
clean_data['RXI_N.B.UK'] = 1 / clean_data['RXI$US_N.B.UK'] 
clean_data['RXI_N.B.JA'] = clean_data['RXI_N.B.JA'] / 100.0

latest = clean_data.iloc[-1]
major = ['RXI_N.B.EU', 'RXI_N.B.UK', 'RXI_N.B.CA', 'RXI_N.B.JA']
oth = ['RXI_N.B.MX', 'RXI_N.B.BZ', 'RXI_N.B.CH', 'RXI_N.B.SD']
twidx = clean_data.resample('MS').mean().append(latest)['JRXWTFB_N.B'].dropna()
clean_data.resample('MS').mean().append(latest)[major].to_csv(data_dir / 'fx1.csv', index_label='date', float_format='%g')
clean_data.resample('MS').mean().append(latest)[oth].to_csv(data_dir / 'fx2.csv', index_label='date', float_format='%g')
twidx.to_csv(data_dir / 'fx_idx.csv', index_label='date', float_format='%g', header=True)

ldate = dtxt(twidx.index[-1])['day1']
lval = twidx.iloc[-1]
totch = ((lval / 100) - 1) * 100
threeyr = twidx.iloc[-38:].mean()
prev3yr = twidx.iloc[-74:-38].mean()

text = (f'The latest index value, as of {ldate}, is {lval:.1f}, an increase '+
        f'of {totch:.1f} percent since inception in 2006. Over the past three years, '+
        f'the index value has averaged {threeyr:.1f}, compared to an average '+
        f'of {prev3yr:.1f} over the previous three-year period.')

write_txt(text_dir / 'twdbasic.txt', text)

text

'The latest index value, as of April 10, 2020, is 122.4, an increase of 22.4 percent since inception in 2006. Over the past three years, the index value has averaged 114.1, compared to an average of 107.5 over the previous three-year period.'

In [14]:
cad = clean_data.loc[ldate, 'RXI_N.B.CA']
jpy = clean_data.loc[ldate, 'RXI_N.B.JA'] * 100
eur = clean_data.loc[ldate, 'RXI_N.B.EU']
gbp = clean_data.loc[ldate, 'RXI_N.B.UK']
cadpc = inc_dec_percent(clean_data['RXI_N.B.CA'].pct_change(262).iloc[-1] * 100)
jpypc = inc_dec_percent(clean_data['RXI_N.B.JA'].pct_change(262).iloc[-1] * 100)
gbppc = inc_dec_percent(clean_data['RXI_N.B.UK'].pct_change(262).iloc[-1] * 100)
eurpc = inc_dec_percent(clean_data['RXI_N.B.EU'].pct_change(262).iloc[-1] * 100)


text = (f'As of {ldate}, one US dollar buys approximately: {cad:.2f} Canadian dollars '+
        f'(see {{\color{{green!85!blue}}\\textbf{{---}}}}), {jpy:.0f} Japanese Yen '+
        f'(see {{\color{{red}}\\textbf{{---}}}}), {eur:.2f} Euros '+
        f'(see {{\color{{cyan!90!white}}\\textbf{{---}}}}), and {gbp:.2f} British Pounds '+
         '(see {\color{blue!90!cyan}\\textbf{---}}). Over the past three years, the nominal '+
        f'exchange rate between the US dollar and the Canadian dollar {cadpc}, the USD-JPY rate '+
        f'{jpypc}, the USD-EUR rate {eurpc}, and the USD-GBP rate {gbppc}.')

write_txt(text_dir / 'selcurr_basic.txt', text)

text

'As of April 10, 2020, one US dollar buys approximately: 1.40 Canadian dollars (see {\\color{green!85!blue}\\textbf{---}}), 108 Japanese Yen (see {\\color{red}\\textbf{---}}), 0.91 Euros (see {\\color{cyan!90!white}\\textbf{---}}), and 0.80 British Pounds (see {\\color{blue!90!cyan}\\textbf{---}}). Over the past three years, the nominal exchange rate between the US dollar and the Canadian dollar increased by 4.8 percent, the USD-JPY rate decreased by 2.3 percent, the USD-EUR rate increased by 2.9 percent, and the USD-GBP rate increased by 4.9 percent.'

### Initial jobless claims

In [15]:
series = 'ICNSA'
start = '1988-01-01'
base = 'https://api.stlouisfed.org/fred/series/observations?'
param = f'series_id={series}&observation_start={start}&api_key={fred_key}'
ftype = '&file_type=json'

url = f'{base}{param}{ftype}'
r = requests.get(url).json()['observations']
data = pd.DataFrame(pd.Series({i['date']: int(i['value']) / 1000 
                               for i in r}).rename('weekly'))

data['1M'] = data['weekly'].rolling(4).mean()

data.to_csv(data_dir / 'icsa.csv', index_label='date', float_format='%g')

totval = data['weekly'].iloc[-1]*1000
datelt = dtxt(data.index[-1])['day1']
latest1m = data["1M"].iloc[-1]*1000
prev1m = data["1M"].iloc[-53]*1000

chval = totval - data['weekly'].iloc[-2]*1000
chtxt = f'{round(abs(chval),-2):,.0f}'
if chval > 1000:
    change = f'a one-week increase of {chtxt}'
elif chval < -1000:
    change = f'a one-week decrease of {chtxt}'
else:
    change = 'virtually unchanged from the previous week'

text = ('The Department of Labor \href{{https://www.dol.gov/ui/data.pdf}}{{reported}} '+
        f'{totval:,.0f} actual new claims for unemployment '+
        f'insurance during the week ending {datelt}, {change}. Over the past four weeks, '+
        f'new claims have averaged {round(latest1m,-2):,.0f} per week. During the same four-'+
        f'week period last year, there were an average of {round(prev1m,-2):,.0f} new claims per week.')

write_txt(text_dir / 'icsa.txt', text)

text

'The Department of Labor \\href{{https://www.dol.gov/ui/data.pdf}}{{reported}} 6,203,359 actual new claims for unemployment insurance during the week ending April 4, 2020, a one-week increase of 187,500. Over the past four weeks, new claims have averaged 3,847,700 per week. During the same four-week period last year, there were an average of 191,100 new claims per week.'

### VIX (SP500 volatility)

In [16]:
prev = pd.read_excel(data_dir/ 'vixarchive.xls', skiprows=1, index_col='Date')
VIX = 'http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vixcurrent.csv'
curr = pd.read_csv(VIX, skiprows=1, index_col='Date', parse_dates=True)
df = (prev.append(curr)['VIX Close'].resample('MS').mean()
      .append(curr['VIX Close'].iloc[-1:]).rename('value'))
df.to_csv(data_dir / 'vix.csv', index_label='date', header='True')

ldate = dtxt(df.index[-1])['day1']
vallt = df.iloc[-1]
val3y = df.iloc[-37:].mean()

compare = compare_text(vallt, val3y, [3, 15, 30])

text = ('This volatility measure, the VIX index (see {\color{magenta}\\textbf{---}}), '+
        f'was {vallt:.1f} on {ldate}, '+
        f'{compare} the average index value of {val3y:.1f} over the '+
        'past three years.')

write_txt(text_dir / 'vixbasic.txt', text)

text

'This volatility measure, the VIX index (see {\\color{magenta}\\textbf{---}}), was 41.2 on April 13, 2020, substantially above the average index value of 17.6 over the past three years.'

### Oil prices (WTI)

In [17]:
df = fred_df('DCOILWTICO')

latest = df.iloc[-1]

ma = df.resample('MS').mean()

p = ma.append(latest)['VALUE']

(p.to_csv(data_dir / 'wti.csv', index_label='date', header=True))

oneyr = p.pct_change(13).iloc[-1] * 100

threeyr = p.pct_change(37).iloc[-1] * 100

oyt = inc_dec_percent(oneyr)
tyt = inc_dec_percent(threeyr)
    
ltch = p.loc['2008-06-01'] - p.iloc[-1]

ldate = dtxt(p.index[-1])['day1']

text = (f'As of {ldate}, a barrel of west Texas '+
        f'intermediate (WTI) \\textbf{{crude oil}} sells for \${p.iloc[-1]:.2f} '+
        '(see {\color{red!80!purple}\\textbf{---}}). Over the past '+
        f'year, this measure of oil prices has {oyt}. Over the past three years, '+
        f'the price {tyt}. Currently, the WTI price is \${ltch:.2f} per barrel '+
        'below its June 2008 average.')

write_txt(text_dir / 'wti.txt', text)

text

'As of April 6, 2020, a barrel of west Texas intermediate (WTI) \\textbf{crude oil} sells for \\$26.21 (see {\\color{red!80!purple}\\textbf{---}}). Over the past year, this measure of oil prices has decreased by 59.0 percent. Over the past three years, the price decreased by 48.7 percent. Currently, the WTI price is \\$107.67 per barrel below its June 2008 average.'

### Inflation Expectations

In [18]:
url = 'http://www.sca.isr.umich.edu/files/tbmpx1px5.csv'
df = pd.read_csv(url)
df.index = pd.to_datetime(df['Month'] + ' ' + df['YYYY'].astype('str'))
data = df['PX5_MD'].loc['2015':]

data.to_csv(data_dir / 'infumich.csv', index_label='date', header='VALUE')

node = end_node(data, 'violet')
write_txt(text_dir / 'infumich_node.txt', node)

data1 = fred_df('T5YIE').loc['2015':,'VALUE']
data1.to_csv(data_dir / 'infbreak.csv', index_label='date', header=True)

node = end_node(data1, 'blue!70!black')
write_txt(text_dir / 'infbreak_node.txt', node)

In [19]:
ldate = dtxt(data.index[-1])['mon1']
pdate = dtxt(data.index[-13])['mon1']
p5val = data.iloc[-61]
lval = data.iloc[-1]
pval = data.iloc[-13]

ldatem = dtxt(data1.index[-1])['day1']
lvalm = data1.iloc[-1]
pdatem = dtxt(data1.index[-262])['day1']
pvalm = data1.iloc[-262]
p5valm = data1.iloc[-(262*5)]

inf_act = pd.read_csv(data_dir / 'cpi.csv')['ALL'].iloc[-60:].mean()

text = (f'As of {ldate}, consumers expect an average inflation rate of {lval} '+
        'percent over the next five years, (see {\color{violet!60!magenta}\\textbf{---}}), '+
        f'compared to an expected rate of {pval} percent '+
        f'in {pdate}. Consumers had expected inflation to average {p5val} percent over the past '+
        f'five years, while actual inflation over the period was {inf_act:.1f} percent.')

write_txt(text_dir / 'inf_exp_cons.txt', text)

text

'As of March 2020, consumers expect an average inflation rate of 2.3 percent over the next five years, (see {\\color{violet!60!magenta}\\textbf{---}}), compared to an expected rate of 2.5 percent in March 2019. Consumers had expected inflation to average 2.8 percent over the past five years, while actual inflation over the period was 1.7 percent.'

In [20]:
text2 = (f'As of {ldatem}, markets expect an average inflation rate of {lvalm} percent ' + 
         'over the next five years (see {\color{blue!70!black}\\textbf{---}}), '
         +f'compared to an expected rate of {pvalm} percent on '+
         f'{pdatem}. Markets had expected inflation to average {p5valm} percent over the '+
         f'past five years, five years ago.')

write_txt(text_dir / 'inf_exp_mkts.txt', text2)

text2

'As of April 13, 2020, markets expect an average inflation rate of 0.94 percent over the next five years (see {\\color{blue!70!black}\\textbf{---}}), compared to an expected rate of 1.86 percent on April 12, 2019. Markets had expected inflation to average 1.6 percent over the past five years, five years ago.'